# ML Experiments usage example

The main goal of ml_experiments is to provide a simple way to track and manage machine learning experiments. It can help by automatically logging parameters through mlflow and by providing a simple interface for different machine learning tasks. The main usage consists in creating a new class inheriting from either BaseExperiment or HPOExperiment, depending on whether we want to perform hyperparameter optimization or not. Let's illustrate a simple example on how to use ml_experiments to train a simple classification model on the iris dataset.

## Creating an experiment

The minimal implementation of any experiment consists on the following methods:

````python
class ClassificationExperiment(BaseExperiment):
    def _add_arguments_to_parser(self):
        raise NotImplementedError

    def _unpack_parser(self):
        raise NotImplementedError

    def _get_combinations_names(self) -> list[str]:
        raise NotImplementedError

    def _get_unique_params(self):
        raise NotImplementedError

    def _get_extra_params(self):
        raise NotImplementedError

    def _load_data(self, combination: dict, unique_params: dict, extra_params: dict, mlflow_run_id: str | None = None, **kwargs):
        raise NotImplementedError

    def _load_model(self, combination: dict, unique_params: dict, extra_params: dict, mlflow_run_id: str | None = None, **kwargs):
        raise NotImplementedError

    def _get_metrics(self, combination: dict, unique_params: dict, extra_params: dict, mlflow_run_id: str | None = None, **kwargs):
        raise NotImplementedError

    def _fit_model(self, combination: dict, unique_params: dict, extra_params: dict, mlflow_run_id: str | None = None, **kwargs):
        raise NotImplementedError

    def _evaluate_model(self, combination: dict, unique_params: dict, extra_params: dict, mlflow_run_id: str | None = None, **kwargs):
        raise NotImplementedError

````


However several other methods are provided to be overridden if needed, they are:
_on_train_start, _before_load_data, _after_load_data, _before_load_model, _after_load_model, _before_get_metrics, _after_get_metrics, _before_fit_model, _after_fit_model, _before_evaluate_model, _after_evaluate_model, _on_exception, _on_train_end

Behind the scenes we call each method one after the other in the following order:
````python

self._on_train_start()
self._before_load_data()
self._load_data()
self._after_load_data()
self._before_load_model()
self._load_model()
self._after_load_model()
self._before_get_metrics()
self._get_metrics()
self._after_get_metrics()
self._before_fit_model()
self._fit_model()
self._after_fit_model()
self._before_evaluate_model()
self._evaluate_model()
self._after_evaluate_model()
self._on_train_end()
````

Each one of these methods takes the same parameters:
````python
def _method_name(self, combination: dict, unique_params: dict, extra_params: dict, mlflow_run_id: str | None = None, **kwargs):
````
where:
- `combination` is a dictionary containing the parameters of the current combination being trained.
- `unique_params` is a dictionary containing the unique parameters of the experiment.
- `extra_params` is a dictionary containing the extra parameters of the experiment.
- `mlflow_run_id` is the ID of the current MLflow run, if any.
- `**kwargs` are the results of the previous methods that can be accessed if needed.

Each method must return a dictionary, this dictionary can be accessed via the `**kwargs` parameter of any subsequent method. This allows to pass data between methods.
The dictionary is accessed by the key `on_<method_name>_return`, so for example imagine we load the data in the `_load_data` method and return it as a dictionary containing the keys `X` and `y`, we can access it in the `_fit_model` method like this:

```python
def _load_data(self, combination: dict, unique_params: dict, extra_params: dict, mlflow_run_id: str | None = None, **kwargs):
	# Load data
	X = ...
	y = ...
	return dict(X=X, y=y)

def _fit_model(self, combination: dict, unique_params: dict, extra_params: dict, mlflow_run_id: str | None = None, **kwargs):
	# Access the loaded data
	load_data_return = kwargs['on_load_data_return']
	X = load_data_return['X']
	y = load_data_return['y']
````

The combination is defined by the `_get_combinations_names` method, which returns a list of strings with the attributes names of the experiment that should be considered as combinations. We will iterate trough the product of these combinations, executing the whole training process for each combination. The combinations parameters are typically the datasets or models configurations (names, seeds, etc). For example, imagine that we are training a `GradientBoostingClassifier` on the iris dataset and we want to split it in different traing and test sets by using different random seeds, besides, we also want to be able to train several models with different n_estimators. In this case, we would define the combinations as follows:

```python
class ClassificationExperiment(BaseExperiment):
	def __init__(
		self,
		*args,
		seed: int | list[int] = 42,
		n_estimators: int | list[int] = 100,
		**kwargs
	):
		super().__init__(*args, **kwargs)
		self.seed = seed
		self.n_estimators = n_estimators

def _get_combinations_names(self) -> list[str]:
	return ['seed', 'n_estimators']
```

This would allow us to train the model with all the combinations of the `seed` and `n_estimators` parameters, for example, if we pass `seed=[42, 43]` and `n_estimators=[100, 200]`, we would train the model with the following combinations:

```
- seed=42, n_estimators=100
- seed=42, n_estimators=200
- seed=43, n_estimators=100
- seed=43, n_estimators=200
```

In the same manner, imagine that we would also like to control the learning rate of the model, but without considering it as a combination, we would define it as a unique parameter:

```python
class ClassificationExperiment(BaseExperiment):
	def __init__(
		self,
		*args,
		seed: int | list[int] = 42,
		n_estimators: int | list[int] = 100,
		learning_rate: float = 0.1,
		**kwargs
	):
		super().__init__(*args, **kwargs)
		self.seed = seed
		self.n_estimators = n_estimators
		self.learning_rate = learning_rate

	def _get_unique_params(self):
		unique_params = super()._get_unique_params()
		unique_params.update({
			'learning_rate': self.learning_rate,
		})
		return unique_params
```

The unique parameters are not considered as combinations, so they will be the same for all the combinations. However, if we change the `learning_rate` parameter this would certainly imply in a different model with different results. Note that we use the super() method to consider the unique_params of the parent class. If the parameter would not influence the results, we could define it as an extra parameter instead. For example, imagine that we want to control the verbosity of the model, but it does not influence the results, we would define it as an extra parameter:

```python
class ClassificationExperiment(BaseExperiment):
	def __init__(
		self,
		*args,
		seed: int | list[int] = 42,
		n_estimators: int | list[int] = 100,
		learning_rate: float = 0.1,
		verbose: bool = False,
		**kwargs
	):
		super().__init__(*args, **kwargs)
		self.seed = seed
		self.n_estimators = n_estimators
		self.learning_rate = learning_rate
		self.verbose = verbose

	def _get_extra_params(self):
		extra_params = super()._get_extra_params()
		extra_params.update({
			'verbose': self.verbose,
		})
		return extra_params
```

The difference between `unique_params` and `extra_params` is that we log both combinations and unique parameters in MLflow, while extra parameters are not logged. 

Finally, for convenience, any experiment can be directly run from the command line, for this we need to implement the `_add_arguments_to_parser` and `_unpack_parser` methods. The first one is used to add the arguments to the argument parser (from the `argparse` module), while the second one is used to unpack the arguments from the parser. For example, we can add the `seed`, `n_estimators`, `learning_rate` and `verbose` parameters as follows:

```python
def _add_arguments_to_parser(self):
	self.parser.add_argument('--seed', type=int, nargs='+', default=[42], help='Random seed for the experiment')
	self.parser.add_argument('--n_estimators', type=int, nargs='+', default=[100], help='Number of estimators for the model')
	self.parser.add_argument('--learning_rate', type=float, default=0.1, help='Learning rate for the model')
	self.parser.add_argument('--verbose', action='store_true', help='Whether to print verbose output')

def _unpack_parser(self):
	args = self.parser.parse_args()
	self.seed = args.seed
	self.n_estimators = args.n_estimators
	self.learning_rate = args.learning_rate
	self.verbose = args.verbose
	return args
```

This allows us to run the experiment from the command line, for example:

```python my_experiment.py --seed 42 43 --n_estimators 100 200 --learning_rate 0.1 --verbose```

The BaseExperiment class already provide several parameters that can be passed either through the command line or as arguments to the constructor, these parameters are:

- `experiment_name`: The name of the experiment, used to create a new MLflow experiment.
- `mlflow_tracking_uri`: The URI of the MLflow tracking server, if not set
- `log_dir`: The directory where the logs will be saved.
- `log_file_name`: The name of the log file.
- `work_root_dir`: The root directory that each training process can use to store intermediate results (fast access) that can be cleared after the training is finished.
- `save_root_dir`: The root directory where the final results will be saved.
- `clean_work_dir`: Whether to clean the `work_dir` (and `work_root_dir` if empty) after the training is finished.
- `raise_on_error`: Whether to raise an exception if an error occurs during the training process or to try to continue with the next combination.
- `parser`: An ArgumentParser instace if it is created outside the experiment class, otherwise it will be created automatically.
- `timeout_fit`: The timeout in seconds for the fitting process, if not set it will wait indefinitely, otherwise it will raise an exception if the fitting process takes longer than this time.
- `timeout_combination`: The timeout in seconds for each combination, if not set it will wait indefinitely, otherwise it will raise an exception if the combination takes longer than this time.
- `verbose`: Whether to print verbose output, this is used to control the verbosity of the logs.
- `profile_time`: Whether to profile the time taken by each method, this is used to measure the time taken by each method and log it in MLflow, we can choose to profile the time of any method by using the decorator @profile_time(enable_based_on_attribute='profile_time') on the method definition.
- `profile_memory`: Whether to profile the memory usage of each method, this is used to measure the memory usage of each method and log it in MLflow, we can choose to profile the memory of any method by using the decorator @profile_memory(enable_based_on_attribute='profile_memory') on the method definition. 

Note that by default, if we not set any of these parameters, the feature will be disabled, so for example, if we do not pass a `mlflow_tracking_uri`, the experiment will not log anything to MLflow, and if we do not pass a `log_dir`, the logs will not be saved to any file.

Also note that we pass the root directory for the work directory and the save directory, so that each training process can use its own work directory, this is useful for parallel execution of the training process. The current `work_dir` and `save_dir` can also be accessed through the kwargs parameter of each method, using the keys 
`work_dir` and `save_dir`, respectively.

Besides, some parameters from the `dask` library are also defined to allow parallel execution of the training process:

- `dask_cluster_type`: The type of Dask cluster to use, can be 'local' or 'slurm'.
- `n_workers`: The number of workers to use in the Dask cluster.
- `n_processes_per_worker`: The number of processes to use per worker in the Dask cluster.
- `n_cores_per_worker`: The number of cores to use per worker in the Dask cluster.
- `n_threads_per_worker`: The number of threads to use per worker in the Dask cluster.
- `n_processes_per_task`: The number of processes used for each task (training process).
- `n_processes_per_task`: The number of processes used for each task (training process).
- `n_cores_per_task`: The number of cores used for each task (training process).
- `n_threads_per_task`: The number of threads used for each task (training process).
- `dask_memory`: The memory limit for each worker in the Dask cluster.
- `dask_job_extra_directives`: Extra directives to be passed to the Dask job when using a Slurm cluster.
- `dask_address`: The address of a Dask cluster if it is already running.
- `n_gpus_per_worker`: The number of GPUs to use per worker, it can be a fraction of a GPU, for example 0.5 means half a GPU.
- `n_gpus_per_task`: The number of GPUs to use per task, it can be a fraction of a GPU, for example 0.5 means half a GPU.

Let's now illustrate the full implementation of the `ClassificationExperiment` class, which will train a `GradientBoostingClassifier` on the iris dataset:



In [1]:
from ml_experiments import BaseExperiment
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score


class ClassificationExperiment(BaseExperiment):
    def __init__(
            self,
            *args,
            seed: int | list[int] = 42,
            n_estimators: int | list[int] = 100,
            learning_rate: float = 0.1,
            model_verbose: int = 1,
            **kwargs
        ):
        super().__init__(*args, **kwargs)
        self.seed = seed
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.model_verbose = model_verbose

    def _add_arguments_to_parser(self):
        self.parser.add_argument(			
            "--seed",
            type=int,
            nargs="+",
            default=self.seed,
            help="Random seed for reproducibility.",
        )
        self.parser.add_argument(
            "--n_estimators",
            type=int,
            nargs="+",
            default=self.n_estimators,
            help="Number of estimators for the model.",
        )
        self.parser.add_argument(
            "--learning_rate",
            type=float,
            default=self.learning_rate,
            help="Learning rate for the model.",
        )
        self.parser.add_argument(
            "--model_verbose",
            type=int,
            default=self.model_verbose,
            help="Verbosity level of the model training.",
            action='store_true'
        )

    def _unpack_parser(self):
        args = super()._unpack_parser()
        self.seed = args.seed
        self.n_estimators = args.n_estimators
        self.learning_rate = args.learning_rate
        self.model_verbose = args.model_verbose

    def _get_combinations_names(self) -> list[str]:
        return ['seed', 'n_estimators'] 

    def _get_unique_params(self):
        unique_params = super()._get_unique_params()
        unique_params.update({
            'learning_rate': self.learning_rate,
        })
        return unique_params

    def _get_extra_params(self):
        extra_params = super()._get_extra_params()
        extra_params.update(
            {
                "model_verbose": self.model_verbose,
            }
        )
        return extra_params

    def _load_data(
        self, combination: dict, unique_params: dict, extra_params: dict, mlflow_run_id: str | None = None, **kwargs
    ):
        seed = combination["seed"]
        iris = load_iris()
        X, y = iris.data, iris.target
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=seed
        )
        return dict(
            X_train=X_train,
            X_test=X_test,
            y_train=y_train,
            y_test=y_test,
        )

    def _load_model(
        self, combination: dict, unique_params: dict, extra_params: dict, mlflow_run_id: str | None = None, **kwargs
    ):
        n_estimators = combination["n_estimators"]
        learning_rate = unique_params["learning_rate"]
        verbose = extra_params["model_verbose"]
        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            verbose=verbose,
        )
        return dict(model=model)

    def _get_metrics(
        self, combination: dict, unique_params: dict, extra_params: dict, mlflow_run_id: str | None = None, **kwargs
    ):
        return dict(accuracy=accuracy_score)

    def _fit_model(
        self, combination: dict, unique_params: dict, extra_params: dict, mlflow_run_id: str | None = None, **kwargs
    ):
        model = kwargs["load_model_return"]["model"]
        X_train = kwargs["load_data_return"]["X_train"]
        y_train = kwargs["load_data_return"]["y_train"]
        model.fit(X_train, y_train)
        return dict()

    def _evaluate_model(
        self, combination: dict, unique_params: dict, extra_params: dict, mlflow_run_id: str | None = None, **kwargs
    ):
        accuracy_fn = kwargs["get_metrics_return"]["accuracy"]
        model = kwargs["load_model_return"]["model"]
        X_test = kwargs["load_data_return"]["X_test"]
        y_test = kwargs["load_data_return"]["y_test"]
        y_pred = model.predict(X_test)
        accuracy = accuracy_fn(y_test, y_pred)
        return dict(accuracy=accuracy)

In [2]:
# We can now run the experiment with the `run` method

experiment = ClassificationExperiment(
	seed=42,
	n_estimators=100,
	learning_rate=0.1,
	model_verbose=0,
)
results = experiment.run(return_results=True)

2025-06-26 16:03:29
Starting experiment...
combination_names: ['seed', 'n_estimators']
combinations: [(42, 100)]
unique_params: {'timeout_fit': None, 'timeout_combination': None, 'learning_rate': 0.1}
extra_params: {'model_verbose': 0}



Combinations completed:   0%|          | 0/1 [00:00<?, ?it/s]

2025-06-26 16:03:29
Running...
seed: 42
n_estimators: 100
timeout_fit: None
timeout_combination: None
learning_rate: 0.1

2025-06-26 16:03:29
Finished!
total_elapsed_time: 0.2398698080005488
seed: 42
n_estimators: 100
timeout_fit: None
timeout_combination: None
learning_rate: 0.1

2025-06-26 16:03:29
Combinations completed:   0%|          | 0/1 [00:00<?, ?it/s]
succesfully_completed: 1
failed: 0
none: 0



In [3]:
# The results are returned as a list of dicts, one for each combination, with the results returned by each method, we can check them with the following convenience method:
from ml_experiments.utils import print_keys
                
print_keys(results[0])  # Print keys of the first result

work_dir
save_dir
load_data_return
  X_train
  X_test
  y_train
  y_test
load_model_return
  model
get_metrics_return
  accuracy
max_memory_used_before_fit
max_memory_used_after_fit
evaluate_model_return
  accuracy
total_elapsed_time
combination
  seed
  n_estimators
unique_params
  timeout_fit
  timeout_combination
  learning_rate
extra_params
  model_verbose
mlflow_run_id
Finished


In [9]:
# The accuracy can be for example accessed like:

accuracy = results[0]["evaluate_model_return"]["accuracy"]
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 1.00


In [10]:
# If we want to run several combinations, we can directly pass them as lists to the constructor:
experiment = ClassificationExperiment(
	seed=[42, 43],
	n_estimators=[100, 200],
	learning_rate=0.1,
	model_verbose=0,
)
results = experiment.run(return_results=True)

2025-06-26 16:05:21
Starting experiment...
combination_names: ['seed', 'n_estimators']
combinations: [(42, 100), (42, 200), (43, 100), (43, 200)]
unique_params: {'timeout_fit': None, 'timeout_combination': None, 'learning_rate': 0.1}
extra_params: {'model_verbose': 0}



Combinations completed:   0%|          | 0/4 [00:00<?, ?it/s]

2025-06-26 16:05:21
Running...
seed: 42
n_estimators: 100
timeout_fit: None
timeout_combination: None
learning_rate: 0.1

2025-06-26 16:05:21
Finished!
total_elapsed_time: 0.2758839699999953
seed: 42
n_estimators: 100
timeout_fit: None
timeout_combination: None
learning_rate: 0.1

2025-06-26 16:05:21
Combinations completed:   0%|          | 0/4 [00:00<?, ?it/s]
succesfully_completed: 1
failed: 0
none: 0

2025-06-26 16:05:21
Running...
seed: 42
n_estimators: 200
timeout_fit: None
timeout_combination: None
learning_rate: 0.1

2025-06-26 16:05:21
Finished!
total_elapsed_time: 0.42614832900017063
seed: 42
n_estimators: 200
timeout_fit: None
timeout_combination: None
learning_rate: 0.1

2025-06-26 16:05:21
Combinations completed:  25%|██▌       | 1/4 [00:00<00:00,  3.44it/s]
succesfully_completed: 2
failed: 0
none: 0

2025-06-26 16:05:21
Running...
seed: 43
n_estimators: 100
timeout_fit: None
timeout_combination: None
learning_rate: 0.1

2025-06-26 16:05:22
Finished!
total_elapsed_time: 0.2

In [12]:
print(f"We have run {len(results)} combinations.")

We have run 4 combinations.


Finally, if we want to automatically log the parameters and results to MLflow, we can pass an `mlflow_tracking_uri`. This will tell the experiment to automatically log the combination, unique_params and the metrics (int or float values) returned by the `_evalute_model` method. It will also try to log every possible parameter of the object `model` if it finds it in the `load_model_retun` dict. Let's illustrate by logging to a sqlite database:

In [13]:
experiment = ClassificationExperiment(
    seed=[42, 43],
    n_estimators=[100, 200],
    learning_rate=0.1,
    model_verbose=0,
    mlflow_tracking_uri="sqlite:///example.db"
)
experiment.run(return_results=False)

2025-06-26 16:08:50
Starting experiment...
combination_names: ['seed', 'n_estimators']
combinations: [(42, 100), (42, 200), (43, 100), (43, 200)]
unique_params: {'timeout_fit': None, 'timeout_combination': None, 'learning_rate': 0.1}
extra_params: {'model_verbose': 0}



Combinations completed:   0%|          | 0/4 [00:00<?, ?it/s]

2025/06/26 16:08:50 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/06/26 16:08:50 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

2025-06-26 16:08:51
Running...
seed: 42
n_estimators: 100
timeout_fit: None
timeout_combination: None
learning_rate: 0.1

2025-06-26 16:08:52
Finished!
total_elapsed_time: 0.22787985600007232
seed: 42
n_estimators: 100
timeout_fit: None
timeout_combination: None
learning_rate: 0.1

2025-06-26 16:08:52
Combinations completed:   0%|          | 0/4 [00:01<?, ?it/s]
succesfully_completed: 1
failed: 0
none: 0

2025-06-26 16:08:52
Running...
seed: 42
n_estimators: 200
timeout_fit: None
timeout_combination: None
learning_rate: 0.1

2025-06-26 16:08:52
Finished!
total_elapsed_time: 0.4372436289995676
seed: 42
n_estimators: 200
timeout_fit: None
timeout_combination: None
learning_rate: 0.1

2025-06-26 16:08:52
Combinations completed:  25%|██▌       | 1/4 [00:02<00:05,  1.95s/it]
succesfully_completed: 2
failed: 0
none: 0

2025-06-26 16:08:52
Running...
seed: 43
n_estimators: 100
timeout_fit: None
timeout_combination: None
learning_rate: 0.1

2025-06-26 16:08:52
Finished!
total_elapsed_time: 0.2

True

We can now check the results of the experiment by running the following mlflow command:
```bash
mlflow ui --backend-store-uri sqlite:///example.db
```

or we can programatically access the results by using the `mlflow` module. Please check the mlflow documentation for more information on how to use it (filter runs, check experiments, etc).

In [19]:
import mlflow

mlflow.set_tracking_uri("sqlite:///example.db")

runs = mlflow.search_runs()
print(f"We have found {len(runs)} runs.")

We have found 4 runs.


In [20]:
runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.max_memory_used_before_fit,metrics.max_memory_used_after_fit,metrics.total_elapsed_time,metrics.accuracy,...,params.min_impurity_decrease,params.log_path,params.cuda_available,params.min_samples_split,params.init,params.n_iter_no_change,params.ccp_alpha,params.learning_rate,tags.mlflow.runName,tags.raised_exception
0,8e8fc054b88b479ca408ce5f56e64860,1,FINISHED,/home/belucci/code/ml_experiments/mlruns/1/8e8...,2025-06-26 19:08:52.997000+00:00,2025-06-26 19:08:53.465000+00:00,330.304,330.304,0.405804,0.933333,...,0.0,None,False,2,None,None,0.0,0.1,kindly-newt-176,False
1,f4679f0956a64c558879c360836413ff,1,FINISHED,/home/belucci/code/ml_experiments/mlruns/1/f46...,2025-06-26 19:08:52.589000+00:00,2025-06-26 19:08:52.891000+00:00,330.304,330.304,0.228453,0.933333,...,0.0,None,False,2,None,None,0.0,0.1,nebulous-duck-591,False
2,5413c635c3dd4657a3e89a34123b215f,1,FINISHED,/home/belucci/code/ml_experiments/mlruns/1/541...,2025-06-26 19:08:52.017000+00:00,2025-06-26 19:08:52.520000+00:00,329.664,330.176,0.437244,1.000000,...,0.0,None,False,2,None,None,0.0,0.1,agreeable-crow-466,False
3,22e01906466448718ad49ee6f54f2962,1,FINISHED,/home/belucci/code/ml_experiments/mlruns/1/22e...,2025-06-26 19:08:51.621000+00:00,2025-06-26 19:08:51.940000+00:00,329.280,329.408,0.227880,1.000000,...,0.0,None,False,2,None,None,0.0,0.1,resilient-wasp-101,False
